# Testing the trained models on images and saving

In [ ]:
# Testing histogram models

import xml.etree.ElementTree as ET
from detecto import core, utils, visualize
import os
import random 
import glob
import matplotlib.pyplot as plt
import cv2
from xml.dom import minidom#, parse
import numpy

from torchvision import transforms
from detecto.utils import normalize_transform

detect_dict = {}

models = glob.glob('/home/travis_s/repos/infloresence_detection/final_set/model_weights_infloresence_2021-07-16_12*')

# model = core.Model.load('/home/travis_s/repos/infloresence_detection/hist_final_set/model_weights_infloresence_2021-07-15_12_epoc_hist_w_step_7_lr_0005.pth', ['flower'])
for model_path in models:
    print(model_path)

    model = core.Model.load(model_path, ['flower'])
    model_broken_path = os.path.basename(model_path).replace('.pth', '').split('_')
    lr = model_broken_path[-1]
    ss = model_broken_path[-3]

    test_cnt = 1
    try:
        for img in glob.glob('./final_set/test/*.png'):
            cnt = 0
            image = utils.read_image(img)
            predictions = model.predict(image)

            # predictions format: (labels, boxes, scores)
            labels, boxes, scores = predictions
            print(labels, scores)
            a_img = cv2.imread(img)
            a_img = cv2.cvtColor(a_img, cv2.COLOR_BGR2RGB)
            copy = a_img.copy()

            xml = img.replace('.png', '.xml')
            mydoc = minidom.parse(xml)
            items = mydoc.getElementsByTagName('object')
            tree = ET.parse(xml)
            root = tree.getroot()

            list_with_all_boxes = []

            for roi in root.iter('object'):
                file_name = root.find('filename').text
                ymin, xmin, ymax, xmax = None, None, None, None 

                ymin = int(roi.find("bndbox/ymin").text)
                xmin = int(roi.find("bndbox/xmin").text)
                ymax = int(roi.find("bndbox/ymax").text)
                xmax = int(roi.find("bndbox/xmax").text)
                gt = [ymin, xmin, ymax, xmax]

                start_point = (xmin, ymax)
                end_point = (xmax, ymin)
                color = (0, 0, 255) 
                thickness = 6
                cv2.rectangle(a_img, start_point, end_point, color, thickness)

            for i, box in enumerate(boxes):

                if scores[i] >= 0.1:
                    cnt += 1
                    min_x, min_y, max_x, max_y = int(box[0]), int(box[1]), int(box[2]), int(box[3])
                    ml = [min_y, min_x, max_y, max_x]

                    start_point = (min_x, max_y)
                    end_point = (max_x, min_y)
                    color = (255, 0, 0) 
                    thickness = 6

                    new_img = copy[min_y:max_y, min_x:max_x, :]
                    cv2.rectangle(a_img, start_point, end_point, color, thickness)

            plt.figure(figsize=(40, 20))
            plt.imshow(a_img) 
            plt.savefig(f'./final_set/learning_rate_{lr}_step_size_{ss}/test_prediction{test_cnt}.png')
            test_cnt += 1  # display it
    except:
        pass